<a href="https://colab.research.google.com/github/Vladimirsp81/DLS_2/blob/master/seq2seq_and_attn_Vladimir_Prudnikov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hausaufgabe
translate(Hausaufgabe) -> **Homework**

Halo!
На семинаре мы создали простую seq2seq модель на основе rnn для перевода, а сейчас постараемся засунуть туда attention. Работать будем с тем же датасетом DE->EN (датасеты получше просто не влезают в память колаба, но если у вас есть CPU+тонна времени или GPU побольше, то можно попробовать построить перевод на WMT14 или IWSLT )

В конце домашней работы предполагается написание отчета о проделанной работе.

In [0]:

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import spacy

import random
import math
import time

from torchtext.datasets import TranslationDataset, Multi30k #WMT14, IWSLT
from torchtext.data import Field, BucketIterator

import torch.nn.functional as F

In [0]:
seed = 43

random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [196]:
! python -m spacy download en
! python -m spacy download de


spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [0]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

# немецкий язык является полем SRC, а английский в поле TRG
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [0]:
# В датасете содержится ~ 30к предложений средняя длина которых 11
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'),  fields = (SRC, TRG))

Давайте посмотрим что у нас с датасетом и сделаем словари для SRC и TGT

In [199]:
labels = ['train', 'validation', 'test']
dataloaders = [train_data, valid_data, test_data]
for d, l in zip(dataloaders, labels):
    print("Number of sentences in {} : {}".format(l, len(d.examples)))


Number of sentences in train : 29000
Number of sentences in validation : 1014
Number of sentences in test : 1000


In [200]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)
print("Number of words in source vocabulary", len(SRC.vocab))
print("Number of words in target vocabulary", len(TRG.vocab))

Number of words in source vocabulary 7855
Number of words in target vocabulary 5893


## Encoder

Энкодер будет ровно как в семинаре, с кдинственным изменением -- forward будет возвращать не только hidden, cell, но еще и outputs. Это нужно (надеюсь, вы уже поняли) для использования attention в декодере

In [0]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        """
        :param: input_dim is the size/dimensionality of the one-hot vectors that will be input to the encoder. This is equal to the input (source) vocabulary size.
        :param: emb_dim is the dimensionality of the embedding layer. This layer converts the one-hot vectors into dense vectors with emb_dim dimensions.
        :param: hid_dim is the dimensionality of the hidden and cell states.
        :param: n_layers is the number of layers in the RNN.
        :param: percentage of the dropout to use
        
        """
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.embedding = nn.Embedding(self.input_dim, self.emb_dim)
 
        self.rnn = nn.LSTM(self.emb_dim, self.hid_dim, self.n_layers)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        """
        :param: src sentences (src_len x batch_size)
        """
        # embedded = <TODO> (src_len x batch_size x embd_dim)
        embedded = self.embedding(src)
        # dropout over embedding
        embedded = self.dropout(embedded)
        outputs, (hidden, cell) = self.rnn(embedded)
        # [Attention return is for lstm, but you can also use gru]
        return outputs, hidden, cell

## Decoder
Оп ля, а тут уже что-то новенькое

Мы будем реализовывать attention, который будет смотреть из tgt в src (НЕ self-attention). 

Определим два класса -- Attention и DecoderAttn. Мы разбили их на два класса, чтобы можно было играться с типом внимания, не меняя код DecoderAttn. Как вы помните с лекции, в качестве аттеншена можно брать любую странную функцию (конкатенация, маленькая сеточка, ...), и все будет работать! Поэтому вам предлагается попробовать несколько разных.


---------------------
Есть два подхода к реализации аттеншена:

Подход #1:

1. Вычисляется embed
2. На основе hidden c прошлого шага, embedded и (возможно) enc_out вычисляется attention, а точнее, веса attention (поэтому не забудьте softmax!!). Размерность batch_size * max_len, max_len -- максимальная длина предложения в батче, т.е. shape[0] от выхода энкодера.
3. К enc_out применяется attention: чаще всего dot product от enc_out и attention_weights (не забудьте про измерение батч. Чтобы нормально вычислить dot_product по батчу, вам поможет torch.bmm)
4. Берутся attention и embedded и сворачиваются в один вектор размерности такой, чтобы кормить его self.lstm. Например, это можно сделать с помощью обычного линейного слоя
5. Вычисляется новое скрытое состояние new_hidden. Это наша self.lstm, примененная к выходу пункта 4.
6. Вычисляется prediction, как в семинаре

Грубо говоря, вся разница с семинаром в том, что мы вместо того, чтобы embedded пихать в self.lstm, миксуем аттэншен на основе всего, что имеем (enc_out, hidden, embedded) и запихиваем в self.lstm микс аттэншена и embedded.

![alt text](https://i.imgur.com/cmkRY0r.png)


Подход #2:

1. Вычисляется embed
2. Вычисляется output, new_hidden (строчка output, (hidden, cell) = self.rnn(embedded, (hidden, cell)))
3. На основе output и enc_out вычисляется attention, а точнее, веса attention (поэтому не забудьте softmax!!)
3. К enc_out применяется attention: чаще всего dot product от enc_out и attention_weights (не забудьте про измерение батч. Чтобы нормально вычислить dot_product по батчу, вам поможет torch.bmm)
4. Вычисляется prediction на основе attention и output. Можно, например, взять nn.Linear() от конкатенации attention и output.

Разница с первым подходом в том, что мы сначала вычисляем выход rnn слоя, а потом смотрим вниманием на src и на основе выхода rnn и attn считаем выход (prediction). 

![alt text](https://i.imgur.com/5aWjQWv.png)


Вам предлагается реализовать хотя бы 1 из вариантов и хотя бы 2 варианта функции attention (в классе Attention)


In [0]:
class Attention(nn.Module):
    def __init__(self, batch_size, hidden_size=None, method="dot"): # add parameters needed for your type of attention
        super(Attention, self).__init__()
        self.method = method # attention method you'll use. e.g. "cat", "one-layer-net", "dot", ...
        self.batch_size = batch_size
        self.hidden_size = hidden_size

        
        if self.method == 'general':
          self.attn = nn.Linear(self.hidden_size, self.hidden_size)
        elif self.method == 'concat':
          self.attn = nn.Linear(self.hidden_size * 2, self.hidden_size)
          self.v = nn.Parameter(torch.FloatTensor(hidden_size)) # Назначаем обучаемый параметр
        
    def dot_score(self, rnn_output, encoder_outputs):
      if self.method == 'dot':
        return torch.sum(rnn_output * encoder_outputs, dim=2)

      elif self.method == 'general':
        energy = self.attn(encoder_outputs)
        return torch.sum(rnn_output * energy, dim=2)

      elif self.method == 'concat':
        # конкатенируем rnn_output и encoder_outputs, применяем линейную трансформацию и регуляризацию
        energy = self.attn(torch.cat((rnn_output.expand(encoder_outputs.size(0), -1, -1), encoder_outputs), 2)).tanh()

        return torch.sum(self.v * energy, dim=2)

    def forward(self, rnn_output, encoder_outputs, seq_len=None):

      score = self.dot_score(rnn_output, encoder_outputs)

      # Применяем softmax для нормализации и дополняем размерностью для возможности перемножения
      attn_weights = F.softmax(score, dim=0).unsqueeze(1)

      context_vector = torch.bmm(attn_weights.transpose(0,2), encoder_outputs.transpose(0,1))

      return context_vector


class DecoderAttn(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, attention, dropout=0.1):
        super(DecoderAttn, self).__init__()
        
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.output_dim = output_dim
        self.n_layers = n_layers
        
        self.attn = attention # instance of Attention class

        # define layers
        self.embedding = nn.Embedding(self.output_dim, self.emb_dim)
        
        self.rnn = nn.LSTM(self.emb_dim, self.hid_dim, self.n_layers) #(lstm embd, hid, layers, dropout?)
        self.out = nn.Linear(self.hid_dim, self.output_dim) # Projection :hid_dim x output_dim
        self.dropout = nn.Dropout(dropout)

        # more layers you'll need for attention
        self.concat = nn.Linear(self.hid_dim * 2, self.hid_dim)
        
    def forward(self, input_, last_hidden, cell, encoder_output):
        # make decoder with attention
        # use code from seminar notebook as base and add attention to it
        input_ = input_.unsqueeze(0)

        # (1 x batch_size x emb_dim)
        embedded = self.embedding(input_) #1 embd over input and dropout
        embedded = self.dropout(embedded)

        # Вычмляем new_hidden
        rnn_output, (new_hidden, cell) = self.rnn(embedded, (last_hidden, cell))

        # Вычисляем вектор контекста (функция forward в Attention)
        context_vector = self.attn(rnn_output, encoder_output)

        # Размеры rnn_output и context_vector -> 1*128*512 и 128*1*512. Удаляем лишние размерности

        rnn_output = rnn_output.squeeze(0)
        context_vector = context_vector.squeeze(1)

        # Конкатенируем и увеличиваем размерность
        concat = torch.cat((context_vector, rnn_output), dim=1)
        concat_out = self.concat(concat)

        # Делаем предсказания, возвращаем результат и состояния
        prediction = self.out(concat_out)

        return prediction, new_hidden, cell

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Seq2Seq module

Здесь опять ничего не поменяется кроме того, что энкодер теперь возвращает свой output, а декодер его принимает

In [0]:
# we need that to put it first to the decoder in 'translate' method
BOS_IDX = SRC.vocab.stoi['<sos>']

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        # Hidden dimensions of encoder and decoder must be equal
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        self._init_weights() 
        self.max_len=30
    
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        """
        :param: src (src_len x batch_size)
        :param: tgt
        :param: teacher_forcing_ration : if 0.5 then every second token is the ground truth input
        """
        
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        # пропускаем источник через энкодер
        enc_out, hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input_ = trg[0, :]
        
        for t in range(1, max_len):
            
            output, hidden, cell = self.decoder(input_, hidden, cell, encoder_output=enc_out) #TODO pass state and input throw decoder 
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            input_ = (trg[t] if teacher_force else top1)
        
        return outputs
    
    def translate(self, src):
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = []
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        src = torch.tensor(src).to(self.device).unsqueeze(1)
        enc_out, hidden, cell = self.encoder(src) # TODO pass src throw encoder
        
        #first input to the decoder is the <sos> tokens
        input_ = torch.tensor([SRC.vocab.stoi['<sos>']] * src.shape[1]).to(device) # TODO trg[idxs]
        
        for t in range(1, self.max_len):
            
            output, hidden, cell = self.decoder(input_, hidden, cell, enc_out) #TODO pass state and input throw decoder 
            top1 = output.max(1)[1]
            outputs.append(top1)
            input_ = (top1)
        
        return outputs
    
    def _init_weights(self):
        p = 0.08
        for name, param in self.named_parameters():
            nn.init.uniform_(param.data, -p, p)

In [0]:
input_dim = len(SRC.vocab)
output_dim = len(TRG.vocab)
src_embd_dim =  tgt_embd_dim = 256
hidden_dim = 512
num_layers =  2
dropout_prob = 0.2

batch_size = 64
PAD_IDX = TRG.vocab.stoi['<pad>']

iterators = BucketIterator.splits((train_data, valid_data, test_data),
                                  batch_size = batch_size, device = device)
train_iterator, valid_iterator, test_iterator = iterators

enc = Encoder(input_dim = input_dim, emb_dim = src_embd_dim, hid_dim=hidden_dim,
              n_layers=num_layers, dropout=dropout_prob)
attn = Attention(batch_size, hidden_dim, method='concat')
dec = DecoderAttn(output_dim=output_dim, emb_dim=tgt_embd_dim, hid_dim=hidden_dim,
                  n_layers=num_layers, attention=attn, dropout=dropout_prob)
model = Seq2Seq(enc, dec, device).to(device)

In [205]:
model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (decoder): DecoderAttn(
    (attn): Attention(
      (attn): Linear(in_features=1024, out_features=512, bias=True)
    )
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2)
    (out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
    (concat): Linear(in_features=1024, out_features=512, bias=True)
  )
)

In [0]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        # print(batch.src.shape)
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing !!
            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)


            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [208]:
%%time
max_epochs = 10
CLIP = 1

# TODO
optimizer = optim.Adam(model.parameters(), lr = 1e-3)
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

best_valid_loss = float('inf')

for epoch in range(max_epochs):
    
    
    train_loss = round(train(model, train_iterator, optimizer, criterion, CLIP), 5)
    valid_loss = round(evaluate(model, valid_iterator, criterion),5)
    
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print('Epoch: {} \n Train Loss {}  Val loss {}:'.format(epoch, train_loss, valid_loss))
    print('Train Perplexity {}  Val Perplexity {}:'.format(np.exp(train_loss), np.exp(valid_loss)))


Epoch: 0 
 Train Loss 4.44562  Val loss 4.3757:
Train Perplexity 85.252718140383  Val Perplexity 79.49546690725073:
Epoch: 1 
 Train Loss 3.21615  Val loss 3.57014:
Train Perplexity 24.931947170479233  Val Perplexity 35.521565822748556:
Epoch: 2 
 Train Loss 2.64126  Val loss 3.33079:
Train Perplexity 14.030871372633799  Val Perplexity 27.960421713637825:
Epoch: 3 
 Train Loss 2.27122  Val loss 3.3077:
Train Perplexity 9.691216889406377  Val Perplexity 27.32221206147416:
Epoch: 4 
 Train Loss 2.02162  Val loss 3.24095:
Train Perplexity 7.5505469171881625  Val Perplexity 25.55799030875298:
Epoch: 5 
 Train Loss 1.82706  Val loss 3.30656:
Train Perplexity 6.21558594697663  Val Perplexity 27.29108248695289:
Epoch: 6 
 Train Loss 1.65348  Val loss 3.33334:
Train Perplexity 5.225131682888519  Val Perplexity 28.031811772648364:
Epoch: 7 
 Train Loss 1.48976  Val loss 3.38329:
Train Perplexity 4.436030743857232  Val Perplexity 29.467560080799277:
Epoch: 8 
 Train Loss 1.3534  Val loss 3.52659

In [209]:
model.load_state_dict(torch.load('model.pt'))
model.eval()

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7855, 256)
    (rnn): LSTM(256, 512, num_layers=2)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (decoder): DecoderAttn(
    (attn): Attention(
      (attn): Linear(in_features=1024, out_features=512, bias=True)
    )
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2)
    (out): Linear(in_features=512, out_features=5893, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
    (concat): Linear(in_features=1024, out_features=512, bias=True)
  )
)

In [210]:
test_loss = evaluate(model, test_iterator, criterion)

print('| Test Loss: {} Test PPL:{}|'.format(test_loss, np.exp(test_loss)))

| Test Loss: 3.2246881872415543 Test PPL:25.145732173154727|


In [0]:
EOS_IDX = SRC.vocab.stoi['<eos>']

def translate(sentence):
    """
    function that uses .translate() method of the model to translate german sentence into english
    params:
        sentence: tokenized gernam sentence
    """
    sentence = sentence.lower()
    sent_vec = [SRC.vocab.stoi[token] for token in sentence.split()]
    
    translation_idx = model.translate(torch.tensor(sent_vec))

    res = []
    for t in translation_idx:
        if t[0] != EOS_IDX:
            # print(TRG.vocab.itos[t[0]], end=' ')
            res.append(TRG.vocab.itos[t[0]])
        else:
            break

    return ' '.join(res)

In [212]:
translate("ein klein apfel")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


'a a field of a band .'

ИИИИ давайте также научимся считать самую популярную метрику для перевода -- BLEU (https://en.wikipedia.org/wiki/BLEU)

В общем-то, вам повезло -- ее писать руками скучно, да и nltk ее написало за вас:

In [0]:
from nltk.translate.bleu_score import corpus_bleu
def compute_bleu(inp_lines, out_lines):
    """ Estimates corpora-level BLEU score of model's translations given inp and reference out """
    translations = [translate(line) for line in inp_lines]
    return corpus_bleu([[ref] for ref in out_lines], translations) * 100

In [214]:
# Получаем батчи тестовых данных, индексы слов в батчах и вычисляем метрику 
bleu = []

for batch in test_iterator:

    src = batch.src.to(device)
    trg = batch.trg.to(device)

    sents_src = [' '.join([SRC.vocab.itos[int(x[i])] for x in src]) for i in range(src.shape[1])]
    sents_trg = [' '.join([TRG.vocab.itos[int(x[i])] for x in trg]) for i in range(trg.shape[1])]

    bleu.append(compute_bleu(sents_src, sents_trg))

print(np.mean(bleu))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


39.80873014489099


Если вы реализовали несколько методов аттеншена, опишите каждый из них и полученную метрику на нем в отчете.

<Место для вашего отчета>

In [0]:
Метод DOT

Для обучения модели по методу dot достаточно малое количество эпох. В текущем задании в ходе экспериментов с количеством эпох и шагом обучения ьыли выбраны следующие праметры:
Количество эпох - 5
Шаг обучения - 1е-3
Размер батча - 64
Как показали эксперименты уменьшение шага обучения приводит к увеличению количества эпох для лучших результатов, но не приводит к росту метрики.
Метрики по методу dot:
Test Loss: 3.3467239886522293
Test PPL: 28.40951114725994
BLEU: 41.643

Метод GENERAL

Наилучшее качество достигается примерно на той же эпохе, требуется больше на 1-2
Увеличение размера батча приводит к снижению метрики. Уменьшение шага обучения не привело к росту метрики.
Метрики по методу ceneral:
Test Loss: 3.2541412860155106
Test PPL: 25.897366552219776
BLEU: 39.77

Метод CONCAT

Выводы аналогичные предыдущим методам.
Метрики по методу concat:
Test Loss: 3.2246881872415543
Test PPL: 25.145732173154727
BLEU: 39.809


Итог: по качеству перевода методы примерно равны. Немного выигрывает DOT. Также он более предпочтителен, т.к. требуется меньшее количество эпох для обучения.